In [92]:
!pip install scikit-learn==0.23.2 --upgrade

     |████████████████████████████████| 538 kB 31.4 MB/s eta 0:00:01


In [130]:
# Importando as bibliotecas que serão utilizadas no deploy do modelo para o Watson Machine Learning
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn.model_selection import train_test_split
import urllib.request
from ibm_watson_machine_learning import APIClient
import sys
import sklearn
import pickle

In [65]:
# Importando o dataset modificado
df_deploy = pd.read_csv("https://raw.githubusercontent.com/VanessaSwerts/locus-data/master/src/datasets/deploy.csv")
df_deploy

,type,area,bedroom,bathroom,garage,latitude,longitude,bathroom_per_bedroom,price
0,Apartamento,73.0,2.0,2.0,2.0,-21.777785,-46.619565,1.000000,285000.0
1,Apartamento,308.0,3.0,1.0,2.0,-21.808696,-46.576769,0.333333,1300000.0
2,Apartamento,60.0,2.0,1.0,2.0,-21.787362,-46.587752,0.500000,240000.0
3,Apartamento,87.0,3.0,1.0,1.0,-21.808696,-46.576769,0.333333,300000.0
4,Casa,500.0,4.0,5.0,3.0,-22.118815,-46.683995,1.250000,1800000.0
...,...,...,...,...,...,...,...,...,...
1783,Casa,330.0,4.0,6.0,4.0,-23.014179,-45.574445,1.500000,930000.0
1784,Apartamento,60.0,2.0,1.0,1.0,-23.675205,-46.432621,0.500000,175000.0
1785,Casa,140.0,3.0,2.0,4.0,-22.885289,-43.328431,0.666667,700000.0
1786,Apartamento,58.0,2.0,1.0,2.0,-21.808696,-46.576769,0.500000,216000.0


In [66]:
# Baixando o modelo compilado do github e salvando no ambiente
urllib.request.urlretrieve("https://raw.githubusercontent.com/VanessaSwerts/locus-data/master/src/model/final_model.sav", "assets/final_model.sav")
model_regression = pickle.load(open('model_final.sav', 'rb'))

('model_final.sav', <http.client.HTTPMessage at 0x7f417a8be580>)

In [68]:
# Testando o modelo final
values_transformed = [[-0.39505918, -0.82530118, 0.09191186, 0.00531812, 0.02077038, -0.06860475, 0.73800738, 1.0, 0.0]]
model_regression.predict(values_transformed)

array([285000.])

In [69]:
# Clonando as transformações customizadas do github
!git clone https://github.com/alexanderaugusto/locus-sklearn-transforms.git

fatal: destination path 'locus-sklearn-transforms' already exists and is not an empty directory.


In [6]:
# Acessando a pasta de customização
!cd locus-sklearn-transforms
!ls -ltr

total 4
drwxrwx--- 5 wsuser wscommon 4096 Nov 21 18:46 locus-sklearn-transforms


In [7]:

# Convertendo a pasta clonada em um arquivo zip
!zip -r locus-sklearn-transforms.zip locus-sklearn-transforms

  adding: locus-sklearn-transforms/ (stored 0%)
  adding: locus-sklearn-transforms/setup.py (deflated 43%)
  adding: locus-sklearn-transforms/setup.cfg (deflated 16%)
  adding: locus-sklearn-transforms/.git/ (stored 0%)
  adding: locus-sklearn-transforms/.git/logs/ (stored 0%)
  adding: locus-sklearn-transforms/.git/logs/HEAD (deflated 29%)
  adding: locus-sklearn-transforms/.git/logs/refs/ (stored 0%)
  adding: locus-sklearn-transforms/.git/logs/refs/remotes/ (stored 0%)
  adding: locus-sklearn-transforms/.git/logs/refs/remotes/origin/ (stored 0%)
  adding: locus-sklearn-transforms/.git/logs/refs/remotes/origin/HEAD (deflated 29%)
  adding: locus-sklearn-transforms/.git/logs/refs/heads/ (stored 0%)
  adding: locus-sklearn-transforms/.git/logs/refs/heads/master (deflated 29%)
  adding: locus-sklearn-transforms/.git/info/ (stored 0%)
  adding: locus-sklearn-transforms/.git/info/exclude (deflated 28%)
  adding: locus-sklearn-transforms/.git/config (deflated 34%)
  adding: locus-sklearn-t

In [83]:
# Instalando a biblioteca customizada do sklearn
!pip install locus-sklearn-transforms.zip

ERROR: Could not find a version that satisfies the requirement locus-sklearn-transforms (from versions: none)
ERROR: No matching distribution found for locus-sklearn-transforms


In [24]:
# Importando a biblioteca customizada do sklearn
from locus_deploy_transforms.sklearn_transformers import CombineNewAttributes

In [48]:
# Dividindo os dados de deploy em features e target
features = ["type", "area", 'bedroom', 'bathroom', "garage", "latitude","longitude", "bathroom_per_bedroom"]
target = ["price"]

X = df_deploy[features]
y = df_deploy[target]

In [49]:
# Separando os dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

In [50]:
# Criando a pipeline de deploy com a transformação customizada e o modelo final
deploy_pipeline = Pipeline(
    steps=[
        ('combine_attrs', CombineNewAttributes(X)),
        ('dtc', model_regression),
    ]
)

In [51]:
# Testando a pipeline de deploy
deploy_pipeline.predict([['Apartamento', 73.0, 2.0, 2.0, 2.0, -21.777785, -46.619565]])

array([285000.])

In [85]:
# Importando as credenciais do IBM Watson Machine Learning
api_key = "pfW4SrxLEOreZPIPY5JijBi5XcnNMQUMa6hbn7P0trxe"
location = "us-south"

wml_credentials = {
    "apikey": api_key,
    "url": 'https://' + location + '.ml.cloud.ibm.com'
}
client = APIClient(wml_credentials)

In [86]:
# Listando os spaces disponíveis no IBM Watson Machine Learning
client.spaces.list()

Note: 'limit' is not provided. Only first 50 records will be displayed if the number of records exceed 50
------------------------------------  -----------------------------------------  ------------------------
ID                                    NAME                                       CREATED
6f2bbe51-1e8d-4aaf-80bc-3cf2bb50b4fa  locus-pricing-calculator-deployment-space  2021-11-15T20:49:24.485Z
------------------------------------  -----------------------------------------  ------------------------


In [87]:
# Setando o space default
client.set.default_space("6f2bbe51-1e8d-4aaf-80bc-3cf2bb50b4fa")

'SUCCESS'

In [88]:
# Verificando as versões do sklearn e python
print('Python version:', sys.version)
print('Sklearn version:', sklearn.__version__)

Python version: 3.8.12 (default, Oct 12 2021, 13:49:34) 
[GCC 7.5.0]
Sklearn version: 0.23.2


In [105]:
# Criando o pacote customizado para inserir a biblioteca customizada do sklearn

pkg_meta = {
   client.package_extensions.ConfigurationMetaNames.NAME: "locus_deploy_transforms",
   client.package_extensions.ConfigurationMetaNames.DESCRIPTION: "A custom sklearn transform",
   client.package_extensions.ConfigurationMetaNames.TYPE: "pip_zip"
}

custom_package_details = client.package_extensions.store(meta_props=pkg_meta, file_path="locus-sklearn-transforms.zip")
custom_package_uid = client.package_extensions.get_uid(custom_package_details)
custom_package_uid

Creating package extensions
SUCCESS


'c634397d-6b64-4ba6-9c0b-8df1744bf43c'

In [122]:
# Criando uma nova especificação de software para armazenar a biblioteca customizada

sofware_spec_uid = client.software_specifications.get_uid_by_name('default_py3.8')

runtime_meta = {
   client.software_specifications.ConfigurationMetaNames.NAME: "skl_pipeline_housing_pricing_prediction",
   client.software_specifications.ConfigurationMetaNames.DESCRIPTION: "A Python runtime to calculate housing princing",
   client.software_specifications.ConfigurationMetaNames.PACKAGE_EXTENSIONS: [{'guid': custom_package_uid}],
   client.software_specifications.ConfigurationMetaNames.SOFTWARE_CONFIGURATION: {},
   client.software_specifications.ConfigurationMetaNames.BASE_SOFTWARE_SPECIFICATION: { 'guid': sofware_spec_uid }
}

runtime_details = client.software_specifications.store(runtime_meta)
custom_runtime_uid = client.software_specifications.get_uid(runtime_details)
custom_runtime_uid

'04f0c8bc-6928-41c8-9c25-ab777254248e'

In [123]:
# Salando o modelo final junto a pipeline de deploy para o IBM Watson Machine Learning

sofware_spec_uid = client.software_specifications.get_uid_by_name('default_py3.8')

metadata = {
    client.repository.ModelMetaNames.NAME: 'Housing Pricing Calculator',
    client.repository.ModelMetaNames.TYPE: 'scikit-learn_0.23',
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID: custom_runtime_uid
}

published_model = client.repository.store_model(
    model=deploy_pipeline,
    meta_props=metadata,
    training_data=None
)

In [124]:
# Listando as versões do modelo
models_details = client.repository.list_models()

------------------------------------  --------------------------  ------------------------  -----------------
ID                                    NAME                        CREATED                   TYPE
a07d5cde-121e-4aa9-a047-ad23d36a2c68  Housing Pricing Calculator  2021-11-21T20:43:43.002Z  scikit-learn_0.23
bc6d727f-02c6-4795-a0b7-9059d87b5185  Housing Pricing Calculator  2021-11-21T19:31:37.002Z  scikit-learn_0.23
9775ca03-49b5-4dc4-b6d1-3fcb37428cb1  Housing Pricing Calculator  2021-11-21T19:30:50.002Z  scikit-learn_0.23
0e04b0af-dc5d-4e23-8d5b-ea1737e3a570  Housing Pricing Calculator  2021-11-21T19:30:41.002Z  scikit-learn_0.23
37d9ae68-d850-4759-9187-71f38ed5c973  Housing Pricing Calculator  2021-11-21T19:27:26.002Z  scikit-learn_0.23
810940ce-1815-4f1a-9b2f-47e94ed677a0  Housing Pricing Calculator  2021-11-21T19:27:07.002Z  scikit-learn_0.23
6e674e95-b706-4695-b578-086bd4db076a  Housing Pricing Calculator  2021-11-20T19:05:44.002Z  scikit-learn_0.23
1c2d2f0a-7e7d-44d5-8eb0

In [125]:
# Fazendo o deploy do modelo

metadata = {
    client.deployments.ConfigurationMetaNames.NAME: "Deployment of Housing Pricing Calculator",
    client.deployments.ConfigurationMetaNames.ONLINE: {}
}

created_deployment = client.deployments.create("a07d5cde-121e-4aa9-a047-ad23d36a2c68", meta_props=metadata)



#######################################################################################

Synchronous deployment creation for uid: 'a07d5cde-121e-4aa9-a047-ad23d36a2c68' started

#######################################################################################


initializing
Note: online_url is deprecated and will be removed in a future release. Use serving_urls instead.
..
ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='b8d06707-6c7f-443a-b8a7-cc5e52f6c496'
------------------------------------------------------------------------------------------------




In [126]:
# Listando os deploys do modelo
client.deployments.list()

------------------------------------  ----------------------------------------  ------  ------------------------
GUID                                  NAME                                      STATE   CREATED
b8d06707-6c7f-443a-b8a7-cc5e52f6c496  Deployment of Housing Pricing Calculator  ready   2021-11-21T20:44:01.639Z
f4c091dd-3913-40a7-8cc2-811b53e5d808  Deployment of Housing Pricing Calculator  failed  2021-11-21T19:31:50.790Z
7cbcd133-4b9f-402d-8056-1d71f719f99a  Deployment of Housing Pricing Calculator  ready   2021-11-21T19:31:04.159Z
0e79b491-5ab2-4e33-a2ad-15609b134f9f  Deployment of Housing Pricing Calculator  failed  2021-11-21T19:27:53.227Z
843154c9-dc9c-4a24-add7-c25dc7427ad8  Deployment of Housing Pricing Calculator  ready   2021-11-20T19:06:00.077Z
58c12b2f-9a17-4a1d-a256-4ee2c15c36d5  Deployment of Housing Pricing Calculator  ready   2021-11-20T19:02:21.631Z
9d9d8944-6331-4eba-af92-176b9fb9291e  Deployment of Housing Pricing Calculator  failed  2021-11-20T19:01:37.037Z


In [129]:
# Testando o modelo final no IBM Watson Machine Learning

values = [['Apartamento', 73.0, 2.0, 2.0, 2.0, -21.777785, -46.619565]]

scoring_payload = {"input_data": [{"values": values }]}
predictions = client.deployments.score("b8d06707-6c7f-443a-b8a7-cc5e52f6c496", scoring_payload)
predictions

{'predictions': [{'fields': ['prediction'], 'values': [[285000.0]]}]}